In [1]:
%load_ext autoreload
%autoreload 2
import warnings; warnings.filterwarnings("ignore")
import pandas as pd
import geopandas as gpd
%matplotlib inline
import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import os
import folium

## Read  CHILE GEM SARA Model (csv format)

In [17]:
path = '/Users/pittore/Documents/PROJECTS/BMBF/RIESGOS/data/GEM_sara_chile/chl-l3-exposure'
infile = 'chl-l3-exposure_total_mod_tax.csv'
sara_data = pd.read_csv(os.path.join(path,infile))

#clean 'name' and use it as ID
sara_data['name'] = sara_data.Region.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
#save all names in an array
sara_names = sara_data['name'].unique()

#group data based on 'name'
datagrp = sara_data.groupby('name')

In [124]:
a = datagrp.get_group("Aisen")
a

,id,Region,Taxonomy,Dwellings,Buildings,Repl-cost-USD-bdg,Population,name
5709,AREA # 11201,Aisén,CR-PC-LWAL-H1-3,78.0,15.6,360000,308.7,Aisen
5710,AREA # 11201,Aisén,CR-LWAL-DNO-H1-3,35.6,8.9,288000,140.8,Aisen
5711,AREA # 11201,Aisén,CR-LWAL-DNO-H4-7,1.7,0.1,1080000,6.7,Aisen
5712,AREA # 11201,Aisén,CR-LWAL-DUC-H1-3,6.7,1.6,336000,26.4,Aisen
5713,AREA # 11201,Aisén,CR-LWAL-DUC-H4-7,1.4,0.1,1260000,5.5,Aisen
5714,AREA # 11201,Aisén,CR-LWAL-DUC-H8-19,1.2,0.0,4032000,4.7,Aisen
5715,AREA # 11201,Aisén,ER-ETR-H1-2,0.6,0.5,43750,2.4,Aisen
5716,AREA # 11201,Aisén,MCF-DNO-H1,3.8,3.7,35000,13.3,Aisen
5717,AREA # 11201,Aisén,MCF-DNO-H1-3,129.4,86.2,94500,508.8,Aisen
5718,AREA # 11201,Aisén,MCF-DUC-H1-3,88.0,22.0,288000,346.5,Aisen


In [145]:

tax_lut = {"W-WS-H1":"W-WS-H1-2",
            "W-WLI-H1":"W-WLI-H1-3",
            "MCF-DNO-H1":"MCF-DNO-H1-3",
            "ER-ETR-H1":"ER-ETR-H1-2",
            "MUR-H1":"MUR-H1-3",
            "MUR-ADO-H1":"MUR-ADO-H1-2",
            "MCF-DNO-H1":"MCF-DNO-H1-3"}

#tax_props = {"W-WS-H1":{"Dwellings": 1,"Repl-cost-USD-bdg":35000,},
#            "W-WLI-H1":{}}

def updategroup(grp):
    group_tax = set(grp.Taxonomy)
    droplist = []
    if 1:
        for i,item in grp.iterrows():
            if item.Taxonomy in tax_lut.keys():
                if tax_lut[item.Taxonomy] in group_tax:
                    #print (tax_lut[item.Taxonomy])
                    tg=grp[grp.Taxonomy==tax_lut[item.Taxonomy]]
                    #print(tg.index)
                    grp.loc[tg.index,"Dwellings"] = tg.Dwellings + item.Dwellings
                    grp.loc[tg.index,"Buildings"] = tg.Buildings + item.Buildings
                    grp.loc[tg.index,"Population"] = tg.Population + item.Population
                    #put index in droplist
                    droplist.append(i)
                else:
                    #just change the name of the taxonomy...
                    grp.loc[i,"Taxonomy"]=tax_lut[item.Taxonomy]

    grp.drop(droplist,inplace=True)
    return(grp)

#testg = datagrp.get_group("Aisen")
#updategroup(testg)

## Read Chile admin_3 areas from GADM

In [10]:
chile_map = '/Users/pittore/Documents/PROJECTS/BMBF/RIESGOS/data/GADM_chile/gadm36_CHL_3.json'
chile_map_gpd = gpd.read_file(chile_map)
chile_map_gpd['name'] = chile_map_gpd.NAME_3.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
len(chile_map_gpd)

302

## Associate exposure model for each admin area and change geometry type to MultiPolygon

In [146]:
from shapely.geometry import MultiPolygon

outmodel = chile_map_gpd[['name','GID_3','geometry']]
outmodel['expo'] = ""
#outmodel.head()
outmodel = outmodel.set_index('name')

for gcname,gc in outmodel.iterrows():
    if (gcname in sara_names):
            testgrp = updategroup(datagrp.get_group(gcname))
            #add initial damage state - by default Null Damage (D0)
            testgrp['Damage'] = 'D0'
            outmodel.at[gcname,'expo'] = testgrp.to_json()
    #convert polygons into multipolygons to avoid errors in file output
    if (gc.geometry.geom_type == 'Polygon'):
        outmodel.at[gcname,'geometry'] = MultiPolygon([gc.geometry])

outmodel.columns = ['gid', 'geometry', 'expo']
outmodel = outmodel.reset_index()
outmodel.head()

,name,gid,geometry,expo
0,Aisen,CHL.1.1.1_1,(POLYGON ((-73.49375152587885 -44.965415954589...,"{""id"":{""5709"":""AREA # 11201"",""5710"":""AREA # 11..."
1,Cisnes,CHL.1.1.2_1,(POLYGON ((-73.31124877929682 -45.111248016357...,"{""id"":{""5729"":""AREA # 11202"",""5730"":""AREA # 11..."
2,Guaitecas,CHL.1.1.3_1,(POLYGON ((-73.49153137207031 -43.711528778076...,"{""id"":{""5747"":""AREA # 11203"",""5748"":""AREA # 11..."
3,Cochrane,CHL.1.2.1_1,(POLYGON ((-72.42082214355469 -47.743259429931...,"{""id"":{""5757"":""AREA # 11301"",""5758"":""AREA # 11..."
4,O'Higgins,CHL.1.2.2_1,(POLYGON ((-73.17006683349609 -49.224048614501...,"{""id"":{""5775"":""AREA # 11302"",""5776"":""AREA # 11..."


### Drop geocells with empty exposure 

In [147]:
blacklist  = outmodel[outmodel.expo ==''].index
outmodel1 = outmodel.drop(blacklist)

In [155]:
#pd.read_json(outmodel1.iloc[2].expo)

## Save to GEOPACKAGE 

In [156]:
#ioschema = gpd.io.file.infer_schema(outmodel)
#ioschema['geometry']='MultiPolygon'
#ioschema
outpath = "/Users/pittore/Documents/workspace/RIESGOS/GFZRIESGOS/updated_schemas/assetmaster"
outfile =os.path.join(outpath,"SARA_v1.0_data.gpkg")
outmodel1.to_file(outfile, driver="GPKG")#,schema = ioschema)

## Display the boundary of the model

In [ ]:
map_osa = folium.Map(location=[-32,-71],tiles='Stamen Terrain',zoom_start=10)
map_osa.choropleth(geo_data=outmodel1,fill_opacity=0.3)
#map_osa.choropleth(geo_data=chile_map,fill_opacity=0.3)
#folium.Marker([-32, -71], popup='test').add_to(map_osa)
map_osa

In [ ]:
#map_osa = folium.Map(location=[df.iloc[event_index].latitude,df.iloc[event_index].longitude])
#map_osm.create_map(path='osm.html')
#map_osa.save('testfolium.html')
#map_osa

## UPDATING THE FRAGILITY MODEL

In [171]:
import json
#tstfile = "/Users/pittore/PowerFolders/riesgos/CODES/WEB_services_and_software/assetmaster/schemas/SARA_v1.0/SARA_v1.0_data.gpkg"
#tst_df = gpd.read_file(outfile,driver="GPKG")
#pd.read_json(tst_df.iloc[0].expo)

tstfile0="/Users/pittore/Documents/workspace/RIESGOS/GFZRIESGOS/updated_schemas/modelprop/SARA_v1.0.json"
jsondata = json.load(open(tstfile0,'r'))
df1 = pd.DataFrame(jsondata["data"])
df1

,taxonomy,D1_mean,D1_stddev,D2_mean,D2_stddev,D3_mean,D3_stddev,D4_mean,D4_stddev,id,imt,imu,im_min,im_max
0,MUR-H1,-1.418,0.310,-0.709,0.328,-0.496,0.322,-0.231,0.317,unique_id,pga,g,0.0,1.0
1,ER-ETR-H1-2,-1.418,0.310,-0.709,0.328,-0.496,0.322,-0.231,0.317,unique_id,pga,g,0.0,1.0
2,MUR-H1-3,-1.418,0.310,-0.709,0.328,-0.496,0.322,-0.231,0.317,unique_id,pga,g,0.0,1.0
3,MUR-ADO-H1-2,-1.782,0.371,-0.894,0.326,-0.644,0.328,-0.335,0.346,unique_id,pga,g,0.0,1.0
4,MUR-STRUB-H1-2,-1.502,0.295,-0.718,0.310,-0.492,0.311,-0.215,0.322,unique_id,pga,g,0.0,1.0
5,MR-DUC-H1-3,-0.532,0.407,0.251,0.394,0.364,0.377,0.578,0.382,unique_id,pga,g,0.0,1.0
6,MCF-DUC-H1-3,-0.532,0.407,0.251,0.394,0.364,0.377,0.578,0.382,unique_id,pga,g,0.0,1.0
7,MCF-DNO-H1-3,-0.804,0.307,-0.046,0.320,0.127,0.314,0.386,0.367,unique_id,pga,g,0.0,1.0
8,MR-DNO-H1-3,-0.804,0.307,-0.046,0.320,0.127,0.314,0.386,0.367,unique_id,pga,g,0.0,1.0
9,CR-LFM-DUC-H1-3,0.088,0.348,0.919,0.398,1.179,0.381,1.616,0.507,unique_id,SA_03,g,0.0,3.0


In [173]:

#tstfile="/Users/pittore/Documents/workspace/RIESGOS/GFZRIESGOS/updated_schemas/modelprop/SARA_v1.0_modelprop.json"
#pd.read_json(tstfile)
#jsondata = json.load(open(tstfile,'r'))
#df = pd.DataFrame(jsondata["data"])
df["D1_2_mean"] = df["D2_mean"]
df["D1_2_stddev"]= df["D2_stddev"]

df["D1_3_mean"] = df["D3_mean"]
df["D1_3_stddev"]= df["D3_stddev"]

df["D1_4_mean"] = df["D4_mean"]
df["D1_4_stddev"]= df["D4_stddev"]

df["D2_3_mean"] = df["D3_mean"]
df["D2_3_stddev"]= df["D3_stddev"]

df["D2_4_mean"] = df["D4_mean"]
df["D2_4_stddev"]= df["D4_stddev"]

df["D3_4_mean"] = df["D4_mean"]
df["D3_4_stddev"]= df["D4_stddev"]

df


,taxonomy,D1_mean,D1_stddev,D2_mean,D2_stddev,D3_mean,D3_stddev,D4_mean,D4_stddev,id,...,D1_3_mean,D1_3_stddev,D1_4_mean,D1_4_stddev,D2_3_mean,D2_3_stddev,D2_4_mean,D2_4_stddev,D3_4_mean,D3_4_stddev
0,MUR-H1,-1.418,0.310,-0.709,0.328,-0.496,0.322,-0.231,0.317,unique_id,...,-0.496,0.322,-0.231,0.317,-0.496,0.322,-0.231,0.317,-0.231,0.317
1,ER_ETR_H1,-1.418,0.310,-0.709,0.328,-0.496,0.322,-0.231,0.317,unique_id,...,-0.496,0.322,-0.231,0.317,-0.496,0.322,-0.231,0.317,-0.231,0.317
2,ER_ETR_H1_2,-1.418,0.310,-0.709,0.328,-0.496,0.322,-0.231,0.317,unique_id,...,-0.496,0.322,-0.231,0.317,-0.496,0.322,-0.231,0.317,-0.231,0.317
3,MUR-H1_3,-1.418,0.310,-0.709,0.328,-0.496,0.322,-0.231,0.317,unique_id,...,-0.496,0.322,-0.231,0.317,-0.496,0.322,-0.231,0.317,-0.231,0.317
4,MUR_ADO_H1_2,-1.782,0.371,-0.894,0.326,-0.644,0.328,-0.335,0.346,unique_id,...,-0.644,0.328,-0.335,0.346,-0.644,0.328,-0.335,0.346,-0.335,0.346
5,MUR-STRUB-H1-2,-1.502,0.295,-0.718,0.310,-0.492,0.311,-0.215,0.322,unique_id,...,-0.492,0.311,-0.215,0.322,-0.492,0.311,-0.215,0.322,-0.215,0.322
6,MUR-STDRE-H1-2,-1.502,0.295,-0.718,0.310,-0.492,0.311,-0.215,0.322,unique_id,...,-0.492,0.311,-0.215,0.322,-0.492,0.311,-0.215,0.322,-0.215,0.322
7,MR_DUC_H1_3,-0.532,0.407,0.251,0.394,0.364,0.377,0.578,0.382,unique_id,...,0.364,0.377,0.578,0.382,0.364,0.377,0.578,0.382,0.578,0.382
8,MCF_DUC_H1_3,-0.532,0.407,0.251,0.394,0.364,0.377,0.578,0.382,unique_id,...,0.364,0.377,0.578,0.382,0.364,0.377,0.578,0.382,0.578,0.382
9,MCF-DNO-H1-3,-0.804,0.307,-0.046,0.320,0.127,0.314,0.386,0.367,unique_id,...,0.127,0.314,0.386,0.367,0.127,0.314,0.386,0.367,0.386,0.367


In [174]:
datastr = df.to_json(orient='records')
jsondata["data"] = eval(datastr)

tstfile2="/Users/pittore/Documents/workspace/RIESGOS/GFZRIESGOS/updated_schemas/modelprop/SARA_v1.0_modelprop.json"

json.dump(jsondata,open(tstfile2,'w'))